In [46]:
from decouple import config
import tweepy
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine

In [43]:
dbuname = config('DBUNAME')
dbpwd = config('DBPWD')
host = config('HOST')
port = '5432'
dbname = 'dbyf1'

In [32]:
bearer_token = config("BEARER_TOKEN")

In [33]:
client = tweepy.Client(bearer_token=bearer_token)

In [47]:
def fetch_load_tweetcount(label, ht1, ht2, start, end, method):
    
    query = '(#bitcoin OR #btc) -is:retweet'
    query = '('+ ht1 + ' OR ' + ht2 + ') -is:retweet'
    print(query)
    start = start
    end = end
    table_name = 'table_'+label
    print(table_name)
        
    while True:    
        #fetch data
        try:
            counts = client.get_recent_tweets_count(query=query,start_time = start, end_time = end , granularity='day')
            tweets_df = pd.DataFrame()

            for count in counts.data:
                tweets_df = tweets_df.append(pd.DataFrame([{'date': count['end'], 
                                                'tweet_count': count['tweet_count']}]))

            tweets_df['date'] = pd.to_datetime(tweets_df['date'])
            tweets_df['label'] = label
            tweets_df = tweets_df.reset_index(drop=True)
            #print(tweets_df)            
        except:
            print("tweet count failed: " + label)
            break
        
                
        #load data
        if_ex_val = method
        
        try:
            conn_string = 'postgresql://'+dbuname+':'+dbpwd+'@'+host+':'+port+'/'+dbname
            engine = create_engine(conn_string)
            tweets_df.to_sql(table_name, conn_string, if_exists = if_ex_val, index=False) 
        except:
            print(e)
            print("Data load failed: " + table_name)   
            break
        break   

In [62]:
hashtags = {'bitcoin': ['#bitcoin','#btc'],'ethereum': ['#eth', '#ethereum'], 'binance': ['#binance','#bnb'],'ripple': ['#ripple','#xrp'],
           'terra':['#terra','#luna'], 'cardano': ['#cardano','#ada'], 'solana': ['#sol','#solana'], 'avalanche': ['#avalanche','#avax'],
           'polkadot':['#polkadot','#dot'],'dogecoin':['#dogecoin', '#doge'], 'msci_world': ['#msciworld', '#msci'],'euro_stoxx': ['#EXW1.DE','#eurostoxx'],
            'smi':['#CSSMI.SW', '#smi'], 'nasdaq': ['#nasdaq', '#EXXT.DE'], 'gold':['#gold','#GC=F'], 'silver':['#silver','#SI=F']}

In [56]:
# Daily load of new data

# get datetime of yesterday and ereyesterday at 00:00:00
yesterday = datetime.strftime(datetime.today() - timedelta(1), "%Y, %m, %d")
ereyesterday = datetime.strftime(datetime.today() - timedelta(2), "%Y, %m, %d")

# set timeframe
end = datetime.strptime(yesterday, "%Y, %m, %d")
start = datetime.strptime(ereyesterday, "%Y, %m, %d")

# set method of function df.to_sql()
method = 'append'

for label in hashtags:
    ht1 = hashtags[label][0]
    ht2 = hashtags[label][1]
    fetch_load_tweetcount(label, ht1, ht2, start, end, method)


(#bitcoin OR #btc) -is:retweet
table_bitcoin
                       date  tweet_count    label
0 2022-04-02 00:00:00+00:00       163049  bitcoin
(#eth OR #ethereum) -is:retweet
table_ethereum
                       date  tweet_count     label
0 2022-04-02 00:00:00+00:00       113450  ethereum
(#binance OR #bnb) -is:retweet
table_binance
                       date  tweet_count    label
0 2022-04-02 00:00:00+00:00       109150  binance
(#ripple OR #xrp) -is:retweet
table_ripple
                       date  tweet_count   label
0 2022-04-02 00:00:00+00:00        46826  ripple
(#terra OR #luna) -is:retweet
table_terra
                       date  tweet_count  label
0 2022-04-02 00:00:00+00:00         3734  terra
(#cardano OR #ada) -is:retweet
table_cardano
                       date  tweet_count    label
0 2022-04-02 00:00:00+00:00        12779  cardano
(#sol OR #solana) -is:retweet
table_solana
                       date  tweet_count   label
0 2022-04-02 00:00:00+00:00        33985  sol

In [64]:
# Initial load of the historical data
yesterday = datetime.strftime(datetime.today() - timedelta(1), "%Y, %m, %d")
firstday = datetime.strftime(datetime.today() - timedelta(6), "%Y, %m, %d")

# set timeframe
end = datetime.strptime(yesterday, "%Y, %m, %d")
start = datetime.strptime(firstday, "%Y, %m, %d")


# set method of function df.to_sql()
method = 'replace'

for label in hashtags:
    ht1 = hashtags[label][0]
    ht2 = hashtags[label][1]
    fetch_load_tweetcount(label, ht1, ht2, start, end, method)

(#bitcoin OR #btc) -is:retweet
table_bitcoin
                       date  tweet_count    label
0 2022-03-29 00:00:00+00:00       152685  bitcoin
1 2022-03-30 00:00:00+00:00       188044  bitcoin
2 2022-03-31 00:00:00+00:00       160642  bitcoin
3 2022-04-01 00:00:00+00:00       152240  bitcoin
4 2022-04-02 00:00:00+00:00       163048  bitcoin
(#eth OR #ethereum) -is:retweet
table_ethereum
                       date  tweet_count     label
0 2022-03-29 00:00:00+00:00        96607  ethereum
1 2022-03-30 00:00:00+00:00       141935  ethereum
2 2022-03-31 00:00:00+00:00       107885  ethereum
3 2022-04-01 00:00:00+00:00       106931  ethereum
4 2022-04-02 00:00:00+00:00       113449  ethereum
(#binance OR #bnb) -is:retweet
table_binance
                       date  tweet_count    label
0 2022-03-29 00:00:00+00:00        83612  binance
1 2022-03-30 00:00:00+00:00       131775  binance
2 2022-03-31 00:00:00+00:00       125602  binance
3 2022-04-01 00:00:00+00:00       104271  binance
4 2022-

2022, 03, 29
